In [2]:




def find_peak(a_list,thred):
    peak_ls=[]
    st_idx=0
    max_idx=0 
    max_range_num=0
    while(st_idx< len(a_list)):
        len_count=0
        while(st_idx < len(a_list) and a_list[st_idx] < thred ):
            st_idx+=1
        while(st_idx < len(a_list) and a_list[st_idx] > thred):
            
            len_count+=1
            if a_list[st_idx] > max_range_num:
                max_idx=st_idx
                max_range_num=a_list[st_idx]
            
            st_idx+=1
        
        max_range_num=0
        if len_count< 80:
            pass 
        else:
            peak_ls.append(max_idx)
    if len(peak_ls)==1 or len(peak_ls)==0:
        return peak_ls
    else:
        rt_ls=[peak_ls[0]]
        
    
    
        for x in peak_ls[1:]:
            
            if x-rt_ls[-1] < 1000:
                if a_list[x] > a_list[rt_ls[-1]]:
                    rt_ls[-1]=x
            else:
                rt_ls.append(x)
    
        return rt_ls
        


def find_action_count(somato_data):

    accumulate_action=0
    for exc in range(6,12):
        fayin_action=somato_data[:,exc]
        real_action=len(find_peak(fayin_action,0.1))
        accumulate_action+=real_action
    
    return accumulate_action

            

In [3]:
"""
trail of one time
"""

import matplotlib.pyplot as plt
import numpy as np
import nengo
from nengo import spa
import time
from tqdm import trange




# dimensions 
d = 64 # change
n = 50*d # change 

# class for delayed connection
delay_t = 0.2   # 200 ms pro Silbe # not change 

class Delay(object):
    def __init__(self, dimensions, timesteps=50): # timesteps change able
        self.timesteps = timesteps
        self.dimensions = dimensions
        self.history = None
        self.reset()
    def step(self, t, x):
        self.history = np.roll(self.history, -self.dimensions)
        self.history[-1] = x
        return self.history[0]
    def reset(self):
        self.history = np.zeros((self.timesteps, self.dimensions))

dt = 0.001
delay = Delay(dimensions=d, timesteps=int(delay_t / dt))

# SPA-Model: 

def nengo_model_set_up(lg,le):
    model = spa.SPA()
    with model:
        # --------------------- SPA representations ----------------

        # High level SPs
        model.visual   = spa.Buffer(dimensions=d)
        model.phonemic = spa.Buffer(dimensions=d)
        model.premotor = spa.Buffer(dimensions=d)
        model.audiexpect = spa.Buffer(dimensions=d)
        model.motor  = spa.Buffer(dimensions=d)
        model.motor1 = nengo.Ensemble(n, dimensions=d)
        model.motor2 = nengo.Ensemble(n, dimensions=d)
        model.somato = spa.Buffer(dimensions=d)

        model.delaynode = nengo.Node(delay.step, size_in=d, size_out=d)

        # High level routing
        actions = spa.Actions(
            'dot(visual, BA) --> phonemic=BA',
            '0.2 --> phonemic=NEUTRAL, premotor=NEUTRAL, audiexpect=NEUTRAL, motor=NEUTRAL',
            'dot(phonemic, BA) --> premotor=BA, motor=BA_EXEC, audiexpect=BA',
            'dot(phonemic, DA) --> premotor=DA, motor=DA_EXEC, audiexpect=DA',
            'dot(phonemic, GA) --> premotor=GA, motor=GA_EXEC, audiexpect=GA',
            'dot(phonemic, PA) --> premotor=PA, motor=PA_EXEC, audiexpect=PA',
            'dot(phonemic, TA) --> premotor=TA, motor=TA_EXEC, audiexpect=TA',
            'dot(phonemic, KA) --> premotor=KA, motor=KA_EXEC, audiexpect=KA',
            #'dot(phonemic, BI) --> premotor=BI, motor=BI_EXEC, audiexpect=BI',
            #'dot(phonemic, DI) --> premotor=DI, motor=DI_EXEC, audiexpect=DI',
            #'dot(phonemic, GI) --> premotor=GI, motor=GI_EXEC, audiexpect=GI',
            #'dot(phonemic, PI) --> premotor=PI, motor=PI_EXEC, audiexpect=PI',
            #'dot(phonemic, TI) --> premotor=TI, motor=TI_EXEC, audiexpect=TI',
            #'dot(phonemic, KI) --> premotor=KI, motor=KI_EXEC, audiexpect=KI',
            #'dot(phonemic, BU) --> premotor=BU, motor=BU_EXEC, audiexpect=BU',
            #'dot(phonemic, DU) --> premotor=DU, motor=DU_EXEC, audiexpect=DU',
            #'dot(phonemic, GU) --> premotor=GU, motor=GU_EXEC, audiexpect=GU',
            #'dot(phonemic, PU) --> premotor=PU, motor=PU_EXEC, audiexpect=PU',
            #'dot(phonemic, TU) --> premotor=TU, motor=TU_EXEC, audiexpect=TU',
            #'dot(phonemic, KU) --> premotor=KU, motor=KU_EXEC, audiexpect=KU',
            #'dot(phonemic, BE) --> premotor=BE, motor=BE_EXEC, audiexpect=BE',
            #'dot(phonemic, DE) --> premotor=DE, motor=DE_EXEC, audiexpect=DE',
            #'dot(phonemic, GE) --> premotor=GE, motor=GE_EXEC, audiexpect=GE',
            #'dot(phonemic, PE) --> premotor=PE, motor=PE_EXEC, audiexpect=PE',
            #'dot(phonemic, TE) --> premotor=TE, motor=TE_EXEC, audiexpect=TE',
            #'dot(phonemic, KE) --> premotor=KE, motor=KE_EXEC, audiexpect=KE',
            #'dot(phonemic, BO) --> premotor=BO, motor=BO_EXEC, audiexpect=BO',
            #'dot(phonemic, DO) --> premotor=DO, motor=DO_EXEC, audiexpect=DO',
            #'dot(phonemic, GO) --> premotor=GO, motor=GO_EXEC, audiexpect=GO',
            #'dot(phonemic, PO) --> premotor=PO, motor=PO_EXEC, audiexpect=PO',
            #'dot(phonemic, TO) --> premotor=TO, motor=TO_EXEC, audiexpect=TO',
            #'dot(phonemic, KO) --> premotor=KO, motor=KO_EXEC, audiexpect=KO',
            'dot(somato, BA_EXEC) --> phonemic=DA',   
            'dot(somato, DA_EXEC) --> phonemic=GA',  
            'dot(somato, GA_EXEC) --> phonemic=PA',  
            'dot(somato, PA_EXEC) --> phonemic=TA',   
            'dot(somato, TA_EXEC) --> phonemic=KA',  
            'dot(somato, KA_EXEC) --> phonemic=BA',  
            #'dot(somato, BI_EXEC) --> phonemic=DI',   
            #'dot(somato, DI_EXEC) --> phonemic=GI',  
            #'dot(somato, GI_EXEC) --> phonemic=PI',  
            #'dot(somato, PI_EXEC) --> phonemic=TI',   
            #'dot(somato, TI_EXEC) --> phonemic=KI',  
            #'dot(somato, KI_EXEC) --> phonemic=BU',  
            #'dot(somato, BU_EXEC) --> phonemic=DU',   
            #'dot(somato, DU_EXEC) --> phonemic=GU',  
            #'dot(somato, GU_EXEC) --> phonemic=PU',  
            #'dot(somato, PU_EXEC) --> phonemic=TU',   
            #'dot(somato, TU_EXEC) --> phonemic=KU',  
            #'dot(somato, KU_EXEC) --> phonemic=BE',  
            #'dot(somato, BE_EXEC) --> phonemic=DE',   
            #'dot(somato, DE_EXEC) --> phonemic=GE',  
            #'dot(somato, GE_EXEC) --> phonemic=PE',  
            #'dot(somato, PE_EXEC) --> phonemic=TE',   
            #'dot(somato, TE_EXEC) --> phonemic=KE',  
            #'dot(somato, KE_EXEC) --> phonemic=BO',  
            #'dot(somato, BO_EXEC) --> phonemic=DO',   
            #'dot(somato, DO_EXEC) --> phonemic=GO',  
            #'dot(somato, GO_EXEC) --> phonemic=PO',  
            #'dot(somato, PO_EXEC) --> phonemic=TO',   
            #'dot(somato, TO_EXEC) --> phonemic=KO',  
            #'dot(somato, KO_EXEC) --> phonemic=BA',  
        )
        nengo.networks.actionselection.Weights.lg = lg
        nengo.networks.actionselection.Weights.le = le
        nengo.networks.actionselection.Weights.wt = 1.0 # change
        nengo.networks.actionselection.Weights.wp = 0.9
        nengo.networks.actionselection.Weights.wp_snr = 0.9 # not used yet

        model.bg = spa.BasalGanglia(actions)
        model.thalamus = spa.Thalamus(model.bg)

        # model delay:
        nengo.Connection(model.motor.state.output, model.motor1, synapse=0.01)

        # no delay:
        #nengo.Connection(model.motor1, model.motor2, synapse=0.01)

        # delay: 
        nengo.Connection(model.motor1, model.delaynode)
        nengo.Connection(model.delaynode, model.motor2)

        nengo.Connection(model.motor2, model.somato.state.input, synapse=0.01)

        # Provide input to high-level
        def start(t):
            if t < 0.25: 
                return 'ZERO'
            elif t < 0.45:
                return 'BA'
            else:
                return 'ZERO'

        model.input = spa.Input(visual=start)

        # set up probes
        visual   = nengo.Probe(model.visual.state.output, synapse=0.03)
        phonemic = nengo.Probe(model.phonemic.state.output, synapse=0.03)
        premotor = nengo.Probe(model.premotor.state.output, synapse=0.03)
        audiexpect = nengo.Probe(model.audiexpect.state.output, synapse=0.03)
        motor = nengo.Probe(model.motor.state.output, synapse=0.03)
        somato = nengo.Probe(model.somato.state.output, synapse=0.03)


    sim = nengo.Simulator(model)
    sim.run(6)

    motor_data=model.similarity(sim.data, motor)
    phonemic_data=model.similarity(sim.data, phonemic)
        
    visual_data=model.similarity(sim.data, visual)
    premotor_data=model.similarity(sim.data, premotor)
    audiexpect_data=model.similarity(sim.data, audiexpect)
    somato_data=model.similarity(sim.data, somato)
        
    return_data_ls=[motor_data,phonemic_data,visual_data,
                        premotor_data,audiexpect_data,somato_data]
        
    accumulate_cnt=find_action_count(motor_data)
    return return_data_ls,accumulate_cnt
        
    

# The premotor ba da ga SPs should give the information concerning all premotor parameters???
# for: targets, execution velocity of point attractors, time delay for starting a point attractor



In [4]:
def find_peak(a_list,thred):
    peak_ls=[]
    st_idx=0
    max_idx=0 
    max_range_num=0
    while(st_idx< len(a_list)):
        len_count=0
        while(st_idx < len(a_list) and a_list[st_idx] < thred ):
            st_idx+=1
        while(st_idx < len(a_list) and a_list[st_idx] > thred):
            
            len_count+=1
            if a_list[st_idx] > max_range_num:
                max_idx=st_idx
                max_range_num=a_list[st_idx]
            
            st_idx+=1
        
        max_range_num=0
        if len_count< 80:
            pass 
        else:
            peak_ls.append(max_idx)
    if len(peak_ls)==1 or len(peak_ls)==0:
        return peak_ls
    else:
        rt_ls=[peak_ls[0]]
        
    
    
        for x in peak_ls[1:]:
            
            if x-rt_ls[-1] < 1000:
                if a_list[x] > a_list[rt_ls[-1]]:
                    rt_ls[-1]=x
            else:
                rt_ls.append(x)
    
        return rt_ls
        


def find_action_count(somato_data):

    accumulate_action=0
    for exc in range(6,12):
        fayin_action=somato_data[:,exc]
        real_action=len(find_peak(fayin_action,0.1))
        accumulate_action+=real_action
    
    return accumulate_action


In [8]:
for idx in range(2,5):
    test_num=11
    lg_range=np.linspace(0.02,0.2,test_num,endpoint=True)
    le_range=np.linspace(0.02,0.2,test_num,endpoint=True)

    
    asw_dict={}
    for i in trange(test_num,  desc='lg loop'):
        lg=lg_range[i]

        for j in trange(test_num, leave=False, unit_scale=True, desc='le loop'):
            le=le_range[j]
            asw_dict[(lg,le)]=nengo_model_set_up(lg,le)


    motor_filename="motor_{}.txt".format(idx)
    

    a_file = open(motor_filename, "w")
    for k,v in asw_dict.items():
        np.savetxt(a_file, v[0][0])


    

lg loop:   0%|          | 0/11 [00:00<?, ?it/s]
le loop:   0%|          | 0.00/11.0 [00:00<?, ?it/s]

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.


le loop:   9%|▉         | 1.00/11.0 [00:56<09:25, 56.6s/it]

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

...

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.


le loop:  91%|█████████ | 10.0/11.0 [09:34<00:57, 57.5s/it]

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.


le loop: 100%|██████████| 11.0/11.0 [10:31<00:00, 57.5s/it]
lg loop: 100%|██████████| 11/11 [1:56:14<00:00, 634.06s/it]


### make figure5 

In [22]:
idx=1
f=open("saved_count{}.txt".format(idx),"w")
for k,v in asw_dict.items():

    f.write(str(k[0])+"_"+str(k[1])+"_"+str(v[1])+"\n")
f.close()

In [8]:
asw_dict[(0.02,0.02)][1]

8

In [9]:
import pandas as pd 
lg_l=[]
le_l=[]
count_l=[]

for k,v in asw_dict.items():
    lg_l.append(k[0])
    le_l.append(k[1])
    count_l.append(v[1])

count_l=[c/max(count_l)* 100 for c in count_l ]
df=pd.DataFrame(data=[lg_l,le_l,count_l])
df=df.T

df.columns=["Y","X","Z"]

fig = plt.figure(figsize=(16,10))
ax = fig.gca(projection='3d')

# to Add a color bar which maps values to colors.
surf=ax.plot_trisurf(df['Y'], df['X'], df['Z'], cmap=plt.cm.coolwarm, linewidth=0.2)
fig.colorbar( surf, shrink=0.5, aspect=5)
#plt.show()
 
# Rotate it
ax.view_init(30, 240)

# Other palette
#ax.plot_trisurf(df['Y'], df['X'], df['Z'], cmap=plt.cm.jet, linewidth=0.01)
#fig.colorbar( surf, shrink=0.5, aspect=5)


plt.show()